<img src="./images/cmiyc2024_test.webp" alt="A kitten with a tophat at a computer. Aka JtR + H" width="800"/>

# Cracking CMIYC 2024 Test Hashes

**GETTING READY FOR CMIYC 2024!!!!**

### Overview

The goal of this notebook is to make it easy as possible for you to get a team registered and ready to participate in CMIYC. I'm hopefull I'll be updating it a lot running up to the contest, so I'd recommend using this as an example, but creating your own notebook to manage your teams cracking sessions. That way you don't have to worry about merge conflicts or overwriting your own work.

### Major Features and Training Goals

1. How to register a team with CMIYC
2. How to decrypt challenge files
3. Still working on this section



### Creating a PGP Key (If Needed)

Let's face it. PGP is a pain, and you don't want to have to try and figure it out during the contest. But all communication with the organizers (KoreLogic) needs to be done via PGP encrypted messages. This includes creating a team and submitting hashes. To make this easier, I created some functions to help using the Python3 **pgpy** library.

The first function is **generate_read_pgp_private_key**. This will create a new PGP key (if one does not already exit) and write it to the key_file name that you send to it. If a PGP key already exists it'll validate the key and then return it. This way you aren't creating a new key every time this is run. Feel free to skip using this function if you already have a PGP key you want to use, but even the conference organizers recommend using a contest specific PGP key to make it easier to partner with other players.

In [1]:
from lib_framework.pgp_mgr import generate_read_pgp_private_key

import os
import sys

# Setting the path this way to make it OS independent
key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'pgp_key.asc'
    )

my_private_key = generate_read_pgp_private_key(key_file, email_address="YOUR_EMAIL_ADDRESS", team_name="YOUR_TEAM_NAME")

### Creating a Registration Email

To make it easier to encrypt/decrypt messages I created the **PGP_Mgr** class. I figured a class would be better than having individual functions since I didn't want to have to keep passing in my private key as well as KoreLogic's public key.

While I could try to integrate this with individual e-mail services, that seems like a lot of work so PGP_Mgr will output encrypted messages to this workbook as well as save the messages to a file. It's up to you then to actually copy/paste (or attach) that message into an e-mail and send it to KoreLogic at sub-2024@contest.korelogic.com

Note, the register_team function will both print the message to your screen as well as return the message. Given the way JupyterLab displays newlines in returned messages I'd recommend hiding the return value using the **";"** at the end of the function call, and copy/pasting the printed version.

In [2]:
from lib_framework.pgp_mgr import PGPMgr, read_pgp_key

# Load up KoreLogic's submisison public key
korelogic_sub_public_key_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'defcon-contest-sub-2024.pub.asc'
    )
korelogic_sub_public_key = read_pgp_key(korelogic_sub_public_key_file)

default_attachment = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'submission.asc'
    )

# Create the PGPMgr class
pgp_mgr = PGPMgr(my_private_key, korelogic_sub_public_key, default_attachment)

# Create Submission e-mail
registration_submission = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'registration_submission.asc'
    )
pgp_mgr.register_team("YOUR_TEAM_NAME", attachment_filename=registration_submission);

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQf/U/cuIP+jESx3rqzGBpI23BNQdihsMRzeBvE0s0/LWipD
lMAVR0AviycG9DqbKDtnIOnnWKLLpHmhn98/kyob0awBetaT1HtSJqgxiO6NOw9M
CHYAghbWP8MaWzAwDbBa9lACFsSJHE0sJOy1Y9BXKai16fd81mYBruivKeAs6YJv
gzmJf18f2EGhi1d5lC9uGsZC42rHqTar0q56u869VP7WtYSuX3UZYUn7t4pDt3qc
x4lTA1fegOZ9iK1bAVhVSU4n2ErHZiurwLoiT3bLUHc2h8MhsD58Zj4mr0g9u8+O
s3pWeFEdFk14bZEemW+Jv9FdK541iUuwQlRsCv/tctLHWwEIQ2aCKw/S+vb4de9Y
76+gkXrv8G8UfIGu8rA/XI82vPn/4i6+zhNKCnkL5H3zTMhlkPb3IKwyk+mSoTQh
/hp7kwb2si42Mhw8e7D3cq8e4JxF8xOZivOew9J5aGEX0v4AmVnTBcNbfVR6B/s+
DAkB4MqHRdlbnpT+vHqxfb9UxCAOe37d1++WCdx/BfeARb23eZss7A+t5nadal5h
1djqVlkHKjuOb5b1R4IwldXHfZWLI8e09P98YHJGUJYfuUW6sSTOInWGWQ9ZQCAz
va6tQpy5AhMTG7Dpib1L1e+fkWiLSuA3nUwXa0qcczHSQYSlvwskB6vWDa7BkJcA
9+X7GWbjD4qarXrUVAtJfx6SDIzEVexAB9sAwwYIXZqeRDID/fXaWcZVBdK+QcO0
UzmA5zOdhGQF3Pvs4S/hTPd8GAJNqC6JTEryJnlWzeq/m+20Z8hbFzzqc9scJ0Gh
rpgy+QP3Kkth3stSRemOSf1zQDLGkQDuz6XOGbkcqMLnCCxltvLfi2o0vx2N6w6C
Q3GEhdZmBKBoLlBJxelRwWNVBxUcikeQt/g7biJ/gYHCfzX3ThzX5Dft256Pe

### Decrypting the KoreLogic Confirmation Email

If your registration submission was sucessfull, then you should get a follow-up e-mail form KoreLogic with a challenge message in it that you will need to decrypt, sign, and send back to them. To do the decryption you can use the **PGPMgr.decrypt_msg() function.

The tricky part is since this isn't integrated directly into your e-mail client, you'll need to save the PGP encrypted part of the message you recieve to a file for this application to read in.

In [3]:
# Now we need to validate the confirmation e-mail
korelogic_challenge_msg = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'korelogic_comms_challenge.asc'
    )
challenge_response = pgp_mgr.decrypt_msg(korelogic_challenge_msg)

Chal: AADD-F342-9A58-0D1C



### Sending Back the Challenge Response to KoreLogic

Once you have the challenge response, you can then encrypt it using the standard **PGPMgr.encrypt_msg() function. This is the function you'll likely be using for most of your conversations with KoreLogic and for submitting your cracked hashes (that is until I automate cracked hash submissions more. This will save the response to a file, which you can then attach to an e-mail and send back to sub-2024@contest.korelogic.com.

Oh and if you are wondering why I'm creating different files for each of these messages, I figured I'd do this in case someone wanted to run this entire notebook so it won't overwrite previous steps. Feel free to use the default filename in the PGPMgr class to only create one file and then run these Jupyter cells one by one for each step.

In [4]:
challenge_response_msg = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'korelogic_comms_response.asc'
    )

pgp_mgr.encrypt_msg(challenge_response, attachment_filename=challenge_response_msg);

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQgAiHkfXJ07kqvFQfDQSp7J1r+xA5P4rWAUGcJJNCa0e1kV
SEUkqk6V3ZSKI7BDR2Q4g0sRV7jE/wwxab0ohYUk9xRMioH8I8RGOjqsRXKF4Q6I
7qGxq8bdrpMOqe5/xcNeL4lcaCidXbXFEUaptxOvKb6pmlWejWxe6W2BWn6xdGfP
helomQWqg4uQDZmZheeuE9i33mNlBzsXpaOPkJR5EhFrRE38x3Oz7YiWpNGaJ6vg
5ycrzxT6Uv6HB2RRqRQVKQGnLjvWuKs/fDlNZIKvrzCzELL1M3Yhn2Y2d1JMrwCS
1i4liki6cFIJdxNX4s08dcTnZ81VrGrsv81NC8OfPNLB2QF+PU/DKHIQSQ0/c0L+
Uy4qhEJVvyoKuysRxsuzEnPPNvSmMwuhimX+9bjghFCkdUyGaKGrJgdWYnfDlzqx
Ja/xePppBdh4lm/5FivGCJjc8ZFbjjthAzx3t2MxXLFdku2fl2njne5/6v5Nto5Y
/DLC4u8ghWHCXe5DnlHxhQnc7gQYtMEyZx34XJggLNCD8mBlhFwY7pVaUR3iyzm+
TO18WCim52q4r5LRl/GS2VomAbqZ6aPgDE0XiQcKUF3nScAGZIBeOxo8zUM5cZOd
HnAeEe36bANw5fKmYvwQ1L2Eic2m0wjLDUlRv7//E0GiQaG4FW001zyzk/QQfTy0
qtvXyk6J79xL0XNKjhkgC2ypT8YOX9Hml3UVqaETIlDMUQM8RDSiPPkr9trFqmMG
+jEclU/39VJINRac3yZP/ZlvQynM80y5BpcWJxWLwWk3qZAZbxAe/orTREa6PImY
NG0c3GGzzswUxpXslDyX1J9vBDyv/jwOIl7fYo8ktuhiw2ViDC02fQy/npUbgqjh
E5k7rI+jXbVTdQgT8hV0fgsreR2bj58GdvkIftC3b/a5oEEzuupsgAb7I1pQX

### Decrypting CMIYC Challenge Files

Now that we have a team, let's get to cracking hashes!!!

The first thing to do is to download the challenge files. You can obtain these from the Korelogic website at [https://contest-2024.korelogic.com/downloads.html](https://contest-2024.korelogic.com/downloads.html). I'd recommend saving it in the challenge file directory, but you really can save the challenge files anywhere and just update the file path to point to it.

The good news is the the challenge files are also encrypted with PGP but this time with a passphrase. So we can use much of the same base code. I created a helper function/wrapper to make this a bit easier. The challenge decryption function is **decrypt_challenge_files(filname, passphrase, attachment_filename)**

The attachment_filename is important since if they give you something like an encyrpted zip file you want to save it as a zip file and not have to deal with pasting raw binary into a file...

In [5]:
from lib_framework.pgp_mgr import decrypt_challenge_files


# Decrypting the first challenge file which is just a text file
challenge_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1_passwd.pgp'
    )

saved_challenge1 = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1_passwd.hash'
    )

# Note, unlike a lot of the other functions, this returns a bytearray and not a string
# This is because I have no idea what Korelogic is going to toss at us
test1_hashes = decrypt_challenge_files(
    challenge_file,
    "mXx/Vtt8K$XHv4_a)4ziF*&CnB$ufLyQUJ=iGh<5",
    attachment_filename=saved_challenge1
    )

print(test1_hashes.decode())

user1:e9fa198e766d7b9bdfdb872032b2c6ee
user2:{SSHA}4Sq+ps0Yeo9HvPV45aeF8Z4wWa1vUlpnTkZIRg==
user3:$1$mjlHQY$7ofnyhSdWnn6HyIMf8GIE0
user4:$1$h2lb97w8$rSnzQB3FHv9o4VXaOTEyx.
user5:$2a$08$SBTibTPJWVGsJzXiaVOzZO9cp8DXeaaQtTCdVe8QxT75N.G2nuk76



In [6]:
# Now we will decrypt the second test challenge file which decrypts to a zip file
challenge_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1.zip.pgp'
    )

saved_challenge2 = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'hashes',
    'test_1.zip'
    )

# Not saving the results into a variable since it's a zip and I don't have time to include
# a python zip extractor into this framework
decrypt_challenge_files(
    challenge_file,
    "&<J>n+MfiY(~Zfg6$:u{t8[&Vh*>iWN;Oo6N;DJ,",
    attachment_filename=saved_challenge2
    );

### Unzip the Zipped Challenge Files

You should have a test_1.zip file in your hashes directory now. You'll need to manually unzip it and copy the challenge files into the hashes directory. 

**Oh wait! The zip itself is password protected. I guess we need to crack that as well :)**

What you'll really need to do is extract the password hash from the encrypted zip file. Luckily I wrote a blog post on how to do this for multiple different file types, so you can reference it during the contest. Here is the link: [https://reusablesec.blogspot.com](https://reusablesec.blogspot.com/2023/11/jupyter-lab-framework-example.html)

Using the John the Ripper Zip2John extension (and specifying the full file path for the zip file and the output since I don't know where you installed JtR), you can use the following command to extract the hash:

`zip2john test_1.zip >> test1.hash`

### A Brief Side-Tangent on Having Multiple Jupyter Notebooks for Different Tasks

Now here is a point where I'm going to recommend that you do something different than what I'm doing in this Notebook. Registering a team and unpacking contest hashes are really one-time events per contest. Having them in a notebook makes them easy to run and debug which is nice, but as we move on to actually cracking hashes we'll start doing tasks which we'll be repeating a lot. Aka loading up and managing POT files, generating new rulesets, analyzing new cracks. Therefore it will add a lot to your quality of life to put those tasks in a different Notebook from the previous one-time "setup" tasks. That way when you reload your crack management notebook you won't have to re-run these one-time tasks that no longer matter.

Now I'm not going to do that since this is more of a tutorial, and splitting up a tutorial into multiple Notebooks will make it easeir to follow and learn from. But just keep that in mind as you set up your own cracking sessions.

### Creating a Config File for the Password Cracking Framework

As we move on to actually cracking passwords, we need to create a config file so that SessionMgr of the Framework will have information about what hash files it needs to load, where to store log files, what is the contest point value for different hash types, etc.

What I normally do for this is copy/paste a previoius config I was using and update the hash info. There is an example copy of this that I've invluded as **/challenge_files/CMIYC2024_Test/config.yml**

Here are a couple of areas that you will need to update:

**Location of Main Potfile and Logs**
```
  jtr_config:
    main_pot_file: "./challenge_files/CMIYC2024_Test/jtr_cmiyc2024.pot"
    log_directory: "./challenge_files/CMIYC2024_Test/"
  
  hashcat_config:
    main_pot_file: "./challenge_files/CMIYC2024_Test/hc_cmiyc2024.potfile"
    log_directory: "./challenge_files/CMIYC2024_Test/"
```

**Values to Assign Cracked Hashes**

**Note**: This is taken from the KoreLogic scoreboard
```
  score_info:
    bcrypt: 16777215
    md5crypt: 65535
    nsldaps: 255
    raw-md5: 1
```

**Files to Load Password Hashes From**

The big question is what format the file is saved as. Common ones in the past have been:
1. "plain_hash" which is just a list of hashes with no metadata associated with them. All the hashes in the list are of the same type, defined in "type".
2. "mixed_list_with_usernames". This is a more complicated one. Basically all the hashes have usernames which need to be parsed and associated with them. Also there are multiple types of hashes that can be found in the file, defined by "hash_types". If you are wondering how to figure out what hash_types are in the file, well that's part of the challenge usually :P

As for the other fields, "source" is helpful since that's saved with the hash metadata to say where the hash came from. This is important as different challenge files often require unique attacks.
```
  challenge_files:
    test1_zip:
      file: "./challenge_files/CMIYC2024_Test/hashes/test1.hash"
      format: "plain_hash"
      type: "pkzip"
      source: "test_zip"
    test1_list:
      file: "./challenge_files/CMIYC2024_Test/hashes/test_1_passwd.hash"
      format: "mixed_list_with_usernames"
      hash_types:
        - bcrypt
        - md5crypt
        - nsldaps
        - raw-md5
      source: "test_list"
```


In [7]:
from lib_framework.session_mgr import SessionMgr

config_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'config.yml'
    )

sm = SessionMgr(config_file)

Starting to load challenge file: ./challenge_files/CMIYC2024_Test/hashes/test1.hash. This may take a minute or two
Done loading the challenge file.
Starting to load challenge file: ./challenge_files/CMIYC2024_Test/hashes/test_1_passwd.hash. This may take a minute or two
Done loading the challenge file.


### Checking Your Progress

The next command tells you what your cracked/uncracked percentage is which can help identify which hash types you should target next.

In [8]:
sm.print_status()

Algorithm      :Total     :Cracked   :Remaining :Percentage
pkzip          :1         :0         :1         :0%
raw-md5        :1         :0         :1         :0%
nsldaps        :1         :0         :1         :0%
md5crypt       :2         :0         :2         :0%
bcrypt         :1         :0         :1         :0%


### Checking Your Score

You can also check your score using the following command. This is helpful since it should "roughly" match what KoreLogic displays as long as you have submitted all of your cracked hashes. If there is a difference in scores, that can tell you something weird is going on so you can look into it.

In [9]:
sm.print_score()

Hash Type:     Value Per Crack:    Points Earned:      Total Possible Points:
pkzip          No Score Defined    0                   0
raw-md5        1                   0                   1
nsldaps        255                 0                   255
md5crypt       65535               0                   131070
bcrypt         16777215            0                   16777215

Total Score: 0
Maximum Possible Score: 16908541


### Displaying Tool Flags for Tracking Cracking Sessions

The following commands highlight command line flags you should pass in to your password cracking programs (aka JtR and HC) to make sure that logs and pot files are saved in the locations this framework is configured to look for them

In [10]:
sm.hc.print_command();
sm.jtr.print_command();

hashcat -o ./challenge_files/CMIYC2024_Test/hc_cmiyc2024.potfile --debug-file ./challenge_files/CMIYC2024_Test/hc_session --debug-mode 5 -p ':'
john --pot=./challenge_files/CMIYC2024_Test/jtr_cmiyc2024.pot --session=./challenge_files/CMIYC2024_Test/jtr_session


### Displaying Hash Format/Types
I'll admit, I never remember the John the Ripper and Hashcat Format/Types for targeting specific Hashes so I'm always looking back at my notes or at their help files when running new cracking sessions. So I created a command to print them out in this framework to make it easy to look up.

In [11]:
sm.print_attack_formats()

Algorithm      :Num Remaining   :JtR Format      :HC Type
pkzip          :1               :pkzip           :17200
raw-md5        :1               :raw-MD5         :0
nsldaps        :1               :Salted-SHA1     :111
md5crypt       :2               :md5crypt        :500
bcrypt         :1               :bcrypt          :3200


### Now Let's Crack Some Hashes

**Enough talk! More Cracking!**

The next step is to actually go about cracking the password hashes. There's a lot of ways to go about doing this, and I've published some of the ones that work for me in previous blogposts on [reusablesec.blogspot.com](reusablesec.blogspot.com). The key is to actually sit down and run some cracking sessions. To look at some unstructured "Eh I want to run something while coding and not think about it too much" attacks I ran you can see the sessions extracted from my JtR logfile below. I'm not proud of these attacks, but at least some of them worked.

**Disclaimer**: I commented this out as the log file was getting much bigger than I really wanted to upload to this repo. I'll need to clean it up and upload it later this week.

In [12]:
# Read all the logfiles stored in the directories specified in the config file
# If you ran your cracking programs using the command lines specified
# above this "should" work

# Note for some JtR logs that can be quite big this can take a long time. So you may occasionally
# want to comment out this if you are repeatedly running this Notebook.
#sm.read_all_logs();

#sm.print_log_sessions()

### Updating Potfiles

The following command will then load cracked passwords from your sessions back into the framework for analysis. You can repeatedly re-run that cell and then re-run other analysis cells to update them as you continue to crack more passwords. For example, you'll likely notice the scores above don't reflect your cracks when you first run this Notebook since we hadn't loaded the potfiles first when displaying them. If you re-run that cell after loading the pots below the score should update to show the hashes you have cracked.

Note: The ServiceMgr.update_main_pots() will sync your potfiles between John the Ripper and Hashcat. This isn't 100% tested, so for the competition it may be worthwhile to periodically create backups of your potfiles.

In [13]:
sm.load_main_pots(verbose=False, update_only=True);
sm.update_main_pots();

Number of new plains added to the JtR pot file: 0
Number of new plains added to the Hashcat pot file: 0


### Analyzing The Results

As of this writing I've cracked 3 of the 6 password hashes, (though the zip one doesn't count for points). The first two hashes I cracked were with the default JtR ruleset (Single Mode + password.lst). The zip password was cracked with a specialty wordlist. So what were those passwords and how did I create that specialty wordlist? Let's look into that...

In [14]:
sm.print_all_plaintext()

HASH TYPE: pkzip ----------------------------------------------------------
Plaintext
When everything happens
HASH TYPE: raw-md5 ----------------------------------------------------------
Plaintext
3-2-1
HASH TYPE: nsldaps ----------------------------------------------------------
Plaintext
Contact
HASH TYPE: md5crypt ----------------------------------------------------------
Plaintext
<No Cracked Hashes Exist For This Category>
HASH TYPE: bcrypt ----------------------------------------------------------
Plaintext
I am a computer at Bell Laboratories and I am learning to talk.


### Continuing the Analysis (How to crack the pkzip file)

I'll admit, the output format isn't great. That's something that I'll hopefully work on later but it certainly won't be done for this competition. But you can see that we cracked two easy hashes with the plaintext:
- 3-2-1
- Contact

If you are an American Gen-Xer or older you may remember that "3-2-1 Contact" was an amazing PBS show back in the day. Minga, the organizer of this conference is an American Gen-Xer and has a preference for old songs. So maybe a wordlist from the theme song might be helpful. Here is a site where we can create that wordlist:

[3-2-1 Contact Songs](https://www.sitcomsonline.com/boards/archive/index.php/t-123444.html)

Using that wordlist I was able to crack the zipfile password which was one of the lyrics: **When everything happens**

**Disclaimer/Acknowledgement**: I'd like to think that I'd have figured this out on my own, but Ivan from team John_Users figured it out first. That's part of the reason why I'm planning on compeating on a streat team instead of with John_Users. I want to force myself to try and figure everything out on my own vs. relying on the really smart people on that team.

### Cracking User5's Password

Once you unzip the file using the cracked passphrase you will find a note that says the following:
___
```
Do not submit the zip passphrase, it is not worth anything.

user5's password is the sentence spoken by the synthesized voice
introducing itself in episode 101.
```
___
We can view all hashes for a particular metadata field, such as the "username" by using the ServiceMgr.create_left_list() function.

**Note**: I just realized that the create_left_list() will not return anything if you have already cracked the password (since it goal is to create a list of hashes you haven't cracked yet). I'll need to look into creating a better way to show that in these writeups. My apologies. You can always delete user5's entry from your potfile to see this example's normal output.

In [15]:
sm.create_left_list(format="jtr", filter={"username":"user5"});

### Cracking User5's Bcrypt

Of course the hard hash is the Bcrypt one! The good news is now I have an excuse to watch old PBS shows. If KoreLogic makes us watch awsome content like this for the contest I won't complain. It's better than listening to Led Zepplin songs!

Here is a link to the episode: [3-2-1 Contact Episode 101](https://archive.org/details/3-2-1-contact-very-first-episode-1980-noisyquiet-production-processing-of-sound-episode-101)

The text in question is: "I am a computer at bell laboratories and I am learning to talk"

It's pretty easy to throw that in a wordlist and try to crack the password. But, it doesn't work.... So there obviously is something wrong with the phrase. Maybe it is capitalization?

One option is to apply passphrase mangling rules from [Passphrase Cracking](https://github.com/initstring/passphrase-wordlist). Besides common passphrase mangling rules this site also has a huge collection of passphrases which is nice for normal cracking session. It'll create mangling rules that generate guesses like:

```
take the red pill
take-the-red-pill
take.the.red.pill
take_the_red_pill
taketheredpill
Take the red pill
TAKE THE RED PILL
tAKE THE RED PILL
Taketheredpill
tAKETHEREDPILL
TAKETHEREDPILL
Take The Red Pill
TakeTheRedPill
Take-The-Red-Pill
Take.The.Red.Pill
Take_The_Red_Pill
```

Unfortunatly this didn't work either. I'll admit this was an area where Ivan from team John_Users also beat me to it. I needed to capititalize Bell Laboratories, and then add a period at the end. That's some of the gotchas that can get you when cracking passwords, and shows I need to up my passphrase mangling rules in the next couple of days!

### Submitting your cracks

This is very much a work in progress part of the Framework, but I'm putting together some helper functions to aid in creating lists to submit to Korelogic. The tricky part is you are not supports to submit the same cracks multiple times, so you need to keep track of what you've submitted previous. The main class to handle this is the **SubMgr**. It will parse through your hash_lists in SessionMgr, and create a list of hash/plaintexts to submit to Korelogic. It'll also help validate confirmation e-mails from KoreLogic, as well as write the successfully submitted hashes to disk so that you can reload this Framework and not have to worry about losing your position.

In [21]:
from lib_framework.submission_mgr import SubmissionMgr

success_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'success_file.sav'
    )

submission_file_prefix = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'sub_backup'
    )

sub_mgr = SubmissionMgr(success_file, submission_file_prefix, sm.hash_list)

### Create the list of hashes:plaintext to submit
The following command will return a string formatted to submit to Korelogic. It'll also create a copy of the plaintext version of this (not encrypted via PGP) to disk to aid in troubleshooting if something goes wrong. You can then feed it into the PGPMgr.encypt_msg() function to create a file to actually submit to KoreLogic. 

In [22]:
# Used to save the PGP message
first_sub_msg = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'first_submit.asc'
    )

# Create the submission and encrypt it
pgp_mgr.encrypt_msg(sub_mgr.create_submission(sm.hash_list), attachment_filename=first_sub_msg);

-----BEGIN PGP MESSAGE-----

wcBMA3EnRIyYyGY/AQf9HEvSFEJSaXrjl+Mvr4+V9ghnNHO0iCzMSmYdoF5Baqld
CdzCxHiBeZZGwjSZ1WAIEEgHZeepkEFhg8cygN9hdS6NV7n7f5Tor2VSuTYW/rVJ
OkQGmy9ckS3fwau/kJ/jycGY65FuVFhOUkyHGxPawb2qgcWU8qDLgAtSN+u2Nh+Q
GkyNRw2m5eouNsjSTpAzHzLOo/kut1/83qigjktbY4tHv7AMrswjKvnbg26nXrKI
FtA5sv4PgFd99YzG9T6RkanK4Q9n82yQ5/ry1TT8pYdBl6YWtktcAWEPiKKgZo3s
P2z9Pk0BTrJBm1GHOQrUTfSju0GoooeGyzVtjsOxyNLBvwFU6zv5THSpweWWADMu
pwED4a1w8Kvz3/NKbB6Xv86mId6xVnwCjzrQVPCnc0RmAGsTBNVqaEAJKMP4XHZ7
82BUgrb25j/S69keyJ9SJeQN5COssoBEd67WjIUK9s6X7i0S8dOtuhc88DUorCig
94Y6hoR8Bvc01FUMHUx8QEkO62kkDYVO/upR0fi2hcalslR/Ut5Vfwci61+9oqVV
8Ri+AI5G/nbJl53pYzfwu6k417qD36gdDVzyq2nIvfhFp50R4tL+n3BjX94PNmIs
49euqNb/N6wZPMlWxp03Tx1S0Uc532C+OONIonRS5T+D5ZR4yQA/VafjTWAPSK53
4vVNFCqSalAsdweG2EL0yvVDq+jf1OiuUM+PihbYsEjqaZdaMj6Ym+zSyE7kwoKj
UnaBl/eq/y6UEanovfvgMkEx/YTp2sW0lt6katYZpFOFatxe+WtuqhA2LzkD6GYa
EehYzCIYdrFi7XDWR9v4ZlpJe+uuqV3LRMNBkKY0/Hmirso6ifPlXvEhce1/neU6
1Bx7Vm+NXd38xmkUzogBtPKGwf07nJtp+L/RCO6THA7VjsHsgf9K4l0aHGhfB

### Processing the Submission Response from KoreLogic
Korelogic will send you an encrypted e-mail saying if there was any problem with your submission. You can decrypt it using the **PGPMgr.decrypt_msg()** function:

In [ ]:
korelogic_sub_response = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2024_Test',
    'korlogic_sub_response.asc'
    )
sub_response = pgp_mgr.decrypt_msg(korelogic_sub_response)

Submission processing results:

Timestamp: 2024-08-04_03:21:28 GMT
New cracks: 3
  Lines received: 3
  Well-formed: 3
  Repeats from previous submissions: 0




### Validating Your Submission
When you submit cracks using this framework, it marks them "in progress". It doesn't actually count that they are submitted until you pass it back the submission response. This is because I want you to be able to catch any weirdness or problems with your submissions. You certainly don't want to go through all the pain of cracking a hash only and not get credit for it! The SubMgr.validate_submission() function also has a flag called **force** which will ignore any errors/discrepencies and acknowledge the results since the class will not let you create another submission until all current "in progress" hashes have been finalized. I did this since it's easy to accidently run these functions multiple times and I want to make sure you don't miss out on sending a submission e-mail. But if things get "wonky" the **force** option can help you get back on track after any manual fixes (such as submitting the hashes yourself without using this framework.

In [ ]:
# Note, set force=True to ignore warnings and acknowledge all in-progress submissions
# I don't recommend this, but it can help if things get wonky
sub_mgr.validate_submission(sub_response, sm.hash_list, force=False);

It looks like this acknowledgment is for an older submission than the current one
Skipping this acknowledgement
